In [1]:
from PIL import Image
import os
import json
import shutil

In [2]:
def is_supported_image(file_path):
  """
  Check if a file has a supported image extension (.png, .jpg, .jpeg).

  Returns:
      bool: True if supported, False otherwise.
  """
  if not file_path.lower().endswith(('.png', '.jpg', '.jpeg')):
    print(f"Skipping {file_path}: not a supported image type")
    return False
  return True

In [3]:
def crop_image_to_aspect(file_path, aspect_ratio, overwrite=False):
  """
  Crop a single image file to the specified aspect ratio (width / height).
  Skips if the image is too short or already fits the ratio.

  Parameters:
      file_path (str): Absolute path to the image file.
      aspect_ratio (float): Desired aspect ratio (width / height), e.g., 4/3.
      overwrite (bool): If True, overwrite the original file. Otherwise, create a new file
                        with '_cropped' appended before the extension.
  """
  if not is_supported_image(file_path):
    return

  filename = os.path.basename(file_path)
  name, ext = os.path.splitext(filename)

  img = Image.open(file_path)
  width, height = img.size
  new_height = int(width / aspect_ratio)

  if new_height >= height:
    print(f"Skipping {filename}: cannot crop ({width}x{height}) to {aspect_ratio:.2f}")
    return

  img_cropped = img.crop((0, 0, width, new_height))

  if overwrite:
    output_path = file_path
  else:
    output_path = os.path.join(os.path.dirname(file_path), f"{name}_cropped{ext}")

  img_cropped.save(output_path)
  print(f"Processed {filename}: {width}x{height} → {width}x{new_height}")

  return os.path.abspath(output_path)

In [4]:
def resize_image_by_width(file_path, width, overwrite=False):
  """
  Resize a single image to the specified width while preserving aspect ratio.
  Skips if the image is already smaller than or equal to the target width.

  Parameters:
      file_path (str): Absolute path to the image file.
      width (int): Target width in pixels.
      overwrite (bool): If True, overwrite the original file. Otherwise, save as a new file
                        with '_resized' appended to the filename.
  """
  if not is_supported_image(file_path):
    return

  filename = os.path.basename(file_path)
  name, ext = os.path.splitext(filename)

  img = Image.open(file_path)
  orig_width, orig_height = img.size

  if orig_width <= width:
    print(f"Skipping {filename}: already ≤ target width ({orig_width}px)")
    return

  new_height = int((width / orig_width) * orig_height)
  img_resized = img.resize((width, new_height), Image.LANCZOS)

  if overwrite:
    output_path = file_path
  else:
    output_path = os.path.join(os.path.dirname(file_path), f"{name}_resized{ext}")

  img_resized.save(output_path, optimize=True, quality=85)

  orig_size_kb = os.path.getsize(file_path) / 1024
  new_size_kb = os.path.getsize(output_path) / 1024

  print(f"Processed {filename}: {orig_width}x{orig_height} → {width}x{new_height}, "
        f"{orig_size_kb:.1f}KB → {new_size_kb:.1f}KB")

  return os.path.abspath(output_path)

In [5]:
def process_all_project_screenshots(project_root, projects_data_file):
  """
  Process all project screenshots.

  Steps:
    1. Copy s.jpg → screenshot.jpg
    2. Crop to a 4/3 aspect ratio
    3. Resize to 400px width
    4. Delete the original s.jpg

  Parameters:
    project_root (str): Directory containing all project folders.
    projects_data_file (str): Path to the JSON file listing project data.
  """
  with open(projects_data_file, "r", encoding="utf-8") as f:
    projects = json.load(f)

  for project in projects:
    slug = project.get("slug")
    project_folder = os.path.join(project_root, slug)
    raw_path = os.path.join(project_folder, "s.jpg")
    screenshot_path = os.path.join(project_folder, "screenshot.jpg")

    if not os.path.isfile(raw_path):
      print(f"\nSkipping {slug}: s.jpg not found")
      continue

    print(f"\nProcessing {slug} screenshot...")

    # Step 1: Copy s.jpg → screenshot.jpg
    shutil.copy2(raw_path, screenshot_path)
    print("Copied s.jpg → screenshot.jpg")

    # Step 2: Crop the screenshot
    crop_image_to_aspect(screenshot_path, aspect_ratio=4/3, overwrite=True)

    # Step 3: Resize the screenshot
    resize_image_by_width(screenshot_path, width=400, overwrite=True)

    # Step 4: Delete the original
    os.remove(raw_path)
    print("Deleted s.jpg")

  print("\n✅ All project screenshots processed successfully!")

In [6]:
process_all_project_screenshots(
    project_root='../',
    projects_data_file='../assets/projects_data.json'
)


Processing yt-cc-extractor screenshot...
Copied s.jpg → screenshot.jpg
Processed screenshot.jpg: 1160x2208 → 1160x870
Processed screenshot.jpg: 1160x870 → 400x300, 19.9KB → 19.9KB
Deleted s.jpg

Skipping loan-calculator: s.jpg not found

Skipping testimonials: s.jpg not found

Skipping image-search-app-2: s.jpg not found

Skipping image-search-app: s.jpg not found

Skipping rem-vs-px: s.jpg not found

Skipping array-methods: s.jpg not found

Skipping wa-layout: s.jpg not found

Skipping flexbox-demo: s.jpg not found

Skipping cgpa-calculator-2: s.jpg not found

Skipping cgpa-calculator: s.jpg not found

Skipping calculator: s.jpg not found

✅ All project screenshots processed successfully!
